## Prediction of Density of States (DOS) using Partial Radial Distribution Function (PRDF) 

We want to study the accuracy and time performance of the featurizations used in [Schutt et al paper](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.89.205118). Here in part 2, we build an ML model and analyze accuracy of the method. 

#### Import packages

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.kernel_ridge import KernelRidge
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.model_selection import cross_val_predict, KFold, GridSearchCV, train_test_split

#### Load featurized data from pickle file

In [2]:
path = os.path.join(os.getcwd(), 'schutt_cutoff16_binsize10.pkl')
data = pd.read_pickle(path)

In [3]:
print ("Shape of data: ", data.shape)
data.head(1)

Shape of data:  (4829, 38644)


,material_id,structure_obj,max_orbital,dos,Cs-Cs PRDF r=0.00-1.00,Cs-Cs PRDF r=1.00-2.00,Cs-Cs PRDF r=2.00-3.00,Cs-Cs PRDF r=3.00-4.00,Cs-Cs PRDF r=4.00-5.00,Cs-Cs PRDF r=5.00-6.00,...,Ne-Ne PRDF r=6.00-7.00,Ne-Ne PRDF r=7.00-8.00,Ne-Ne PRDF r=8.00-9.00,Ne-Ne PRDF r=9.00-10.00,Ne-Ne PRDF r=10.00-11.00,Ne-Ne PRDF r=11.00-12.00,Ne-Ne PRDF r=12.00-13.00,Ne-Ne PRDF r=13.00-14.00,Ne-Ne PRDF r=14.00-15.00,Ne-Ne PRDF r=15.00-16.00
0,mp-85,[[0. 0. 0.] In],spd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Count sp and spd systems

In [4]:
count = data['max_orbital'].value_counts(sort=True)
count

spd    4600
sp      229
Name: max_orbital, dtype: int64

Partition compounds into $sp$ and $spd$ systems.

In Schutt's paper, there are 1716 sp systems and 5548 spd systems.

In [5]:
data = data.sort_values(by=['max_orbital'])

In [6]:
sp_data = data[:count['sp']]
spd_data = data[count['sp']:]
print ("sp systems: ", sp_data.shape[0])
print ("spd systems: ", spd_data.shape[0])

sp systems:  229
spd systems:  4600


Input X and Y

In [7]:
x_sp = sp_data.drop(['material_id', 'structure_obj', 'max_orbital', 'dos'], 1)
y_sp = sp_data['dos']
x_sp, y_sp = shuffle(x_sp, y_sp)

In [8]:
x_spd = spd_data.drop(['material_id', 'structure_obj', 'max_orbital', 'dos'], 1)
y_spd = spd_data['dos']
x_spd, y_spd = shuffle(x_spd, y_spd)

Partition data into training and testing set (80/20)

In [9]:
x_train, x_test, y_train, y_test = dict.fromkeys(['sp', 'spd']), dict.fromkeys(['sp', 'spd']), dict.fromkeys(['sp', 'spd']), dict.fromkeys(['sp', 'spd'])
x_train['sp'], x_test['sp'], y_train['sp'], y_test['sp'] = train_test_split(x_sp, y_sp, test_size=0.2, shuffle=True)
x_train['spd'], x_test['spd'], y_train['spd'], y_test['spd'] = train_test_split(x_spd, y_spd, test_size=0.2, shuffle=True)

### Build ML model

Set up Kernel Ridge Regression (KRR) model. Three types of kernel: linear, Gaussian, Laplacian

In [10]:
kernels = ['linear', 'gaussian', 'laplacian']

In [11]:
sp_models = {"linear": KernelRidge(kernel="linear"), "gaussian": KernelRidge(kernel="rbf"), "laplacian": KernelRidge(kernel="laplacian")}
spd_models = {"linear": KernelRidge(kernel="linear"), "gaussian": KernelRidge(kernel="rbf"), "laplacian": KernelRidge(kernel="laplacian")}

Here we'll compare the performance of three different kernels in predicting DOS for sp and spd systems.

In [ ]:
kfold = KFold(5)

In [ ]:
cv_prediction_sp, cv_prediction_spd = dict.fromkeys(kernels), dict.fromkeys(kernels)

In [ ]:
for model in sp_models:
    sp_models[model] = sp_models[model].fit(x_train['sp'], y_train['sp'])
    cv_prediction_sp[model] = sp_models[model].predict(x_test['sp'])
#     cv_prediction_sp[model] = cross_val_predict(sp_models[model], x_test['sp'], y_test['sp'], cv=kfold)

In [ ]:
for model in spd_models:
    spd_models[model] = spd_models[model].fit(x_train['spd'], y_train['spd'])
    cv_prediction_spd[model] = spd_models[model].predict(x_test['spd'])
#     cv_prediction_spd[model] = cross_val_predict(spd_models[model], x_test['spd'], y_test['spd'], cv=kfold)

Compute aggregate statistics of the kernels used.

In [ ]:
stats = ['mean_absolute_error', 'mean_squared_error', 'r2_score']
score_sp, score_spd = dict.fromkeys(kernels), dict.fromkeys(kernels)
for i, j in zip(score_sp, score_spd):
    score_sp[i], score_spd[j] = dict.fromkeys(stats), dict.fromkeys(stats)

In [ ]:
for model in kernels:
    for scorer in stats:
        score_sp[model][scorer] = getattr(metrics, scorer)(y_test['sp'], cv_prediction_sp[model])
        score_spd[model][scorer] = getattr(metrics, scorer)(y_test['spd'], cv_prediction_spd[model])

In [ ]:
print ("sp system score: ")
for i in score_sp:
    print (i, ": ", score_sp[i])
print ()
print ("spd system score: ")
for i in score_spd:
    print (i, ": ", score_spd[i])

Plot prediction

In [ ]:
max(cv_prediction_sp['linear'])

In [ ]:
fig, ax = plt.subplots(2, 3, sharex=True, sharey=True)

ax[0, 0].set_title("Linear")
ax[0, 1].set_title("Gaussian")
ax[0, 2].set_title("Laplacian")

ax[0, 0].set_ylabel("sp\n $DOS_{pred} (states/eV/A^3)$")
ax[1, 0].set_ylabel("spd\n $DOS_{pred} (states/eV/A^3)$")
ax[1, 1].set_xlabel("$DOS_{calc} (states/eV/A^3)$")

ax[0, 0].scatter(y_test['sp'], cv_prediction_sp['linear'], marker='.')
ax[0, 1].scatter(y_test['sp'], cv_prediction_sp['gaussian'], marker='.')
ax[0, 2].scatter(y_test['sp'], cv_prediction_sp['laplacian'], marker='.')

ax[1, 0].scatter(y_test['spd'], cv_prediction_spd['linear'], marker='.')
ax[1, 1].scatter(y_test['spd'], cv_prediction_spd['gaussian'], marker='.')
ax[1, 2].scatter(y_test['spd'], cv_prediction_spd['laplacian'], marker='.')

fig.tight_layout()
fig.set_size_inches((10, 7))

# ax1.text(0.49, 0.026, 'MAE: {:.0f}\nRMSE:{:.0f}\n$R^2$: {:.3f}'.format(score['sp']['mean_absolute_error'], score['sp']['mean_squared_error'], score['sp']['r2_score']),
#          transform=ax1.transAxes, fontsize=8,
#          bbox={'facecolor': 'w', 'edgecolor': 'k'})

Using GridSearchCV to do hyperparameter searching

In [12]:
params = {}
params['prdf'] = [{'alpha' : [10**(-a) for a in range(2,6,2)],
    'gamma': [1/2.0/s/s for s in (20000,40000,80000,160000)]}]

In [13]:
gridsearch_sp = GridSearchCV(KernelRidge(kernel='laplacian'), params['prdf'], cv=KFold(), refit=True, n_jobs=-1)
gridsearch_spd = GridSearchCV(KernelRidge(kernel='laplacian'), params['prdf'], cv=KFold(), refit=True, n_jobs=-1)

In [14]:
gridsearch_sp.fit(x_train['sp'], y_train['sp'])

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
       error_score='raise',
       estimator=KernelRidge(alpha=1, coef0=1, degree=3, gamma=None, kernel='laplacian',
      kernel_params=None),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'alpha': [0.01, 0.0001], 'gamma': [1.25e-09, 3.125e-10, 7.8125e-11, 1.953125e-11]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [15]:
gridsearch_spd.fit(x_train['spd'], y_train['spd'])

Process ForkPoolWorker-15:
Process ForkPoolWorker-16:
Process ForkPoolWorker-13:
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/AikRui/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/AikRui/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/AikRui/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/AikRui/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/AikRui/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/AikRui/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/AikRui/

KeyboardInterrupt: 

In [ ]:
best_model = {'sp': gridsearch_sp.best_estimator_, 'spd': gridsearch_spd.best_estimator_}

In [ ]:
cv_predictions = {'sp': best_model['sp'].predict(x_test['sp']), 'spd': best_model['spd'].predict(x_test['spd'])}

In [ ]:
stats = {'sp': {'mae': mean_absolute_error(y_test['sp'], cv_predictions['sp']),
                'rmse': np.sqrt(mean_squared_error(y_test['sp'], cv_predictions['sp'])),
                'r2': r2_score(y_test['sp'], cv_predictions['sp'])}, 
         'spd': {'mae': mean_absolute_error(y_test['spd'], cv_predictions['spd']),
                'rmse': np.sqrt(mean_squared_error(y_test['spd'], cv_predictions['spd'])),
                'r2': r2_score(y_test['spd'], cv_predictions['spd'])}}

In [ ]:
print ("sp system score: ")
for k, v in stats.items():
    print (i, ":\n", v)